In [1]:
import os
from string import Template
from collections import namedtuple

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import sys
sys.path.insert(0, '..')
from src.data import TimeSeries
from src.methods.deep import DeepStoppingModel
from src.eval import (
    mean_time_from_event,
    classification_metrics
)

/Users/daltonschutte/dev/practicum/env/lib/python3.11/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
br_train = TimeSeries.from_csv(
    'pandas',
    '../data/blood-refrigerator/train.csv'
)
br_dev = TimeSeries.from_csv(
    'pandas',
    '../data/blood-refrigerator/val.csv'
)
br_test = TimeSeries.from_csv(
    'pandas',
    '../data/blood-refrigerator/test.csv'
)
print(br_train.shape, br_dev.shape, br_test.shape)
br_dev.head()

(60166, 14) (13992, 14) (65763, 14)


,timestamp,Product temperature base [°C],Evaportator temperature base [°C],Power supply [V],Condenser temperature base [°C],Instant power consumption [W],Signal [DBM],Door_alert,Door_close,Door_open,Machine_cooling,Machine_defrost,Machine_pause,PW_0.5h
0,2022-11-24 17:38:58.399500,0.677748,0.354826,-0.321659,-0.785476,-0.685568,-1.45181,0.0,1.0,0.0,1.0,0.0,0.0,0
1,2022-11-24 17:39:32.544000,0.677748,0.354826,-0.321659,-0.785476,-0.685568,-1.45181,0.0,1.0,0.0,1.0,0.0,0.0,0
2,2022-11-24 17:40:06.688500,0.677748,-0.629940,0.214894,0.093322,-0.685568,-1.45181,0.0,1.0,0.0,0.0,0.0,1.0,0
3,2022-11-24 17:40:40.833000,0.677748,-0.629940,0.214894,0.093322,-0.685568,-1.45181,0.0,1.0,0.0,0.0,0.0,1.0,0
4,2022-11-24 17:41:14.977500,0.677748,-0.629940,0.214894,0.093322,-0.685568,-1.45181,0.0,1.0,0.0,0.0,0.0,1.0,0


In [3]:
br_train.parse_datetime('timestamp')
br_dev.parse_datetime('timestamp')
br_test.parse_datetime('timestamp')

br_train.split_by_day()
br_dev.split_by_day()
br_test.split_by_day()

len(br_train.time_series), len(br_dev.time_series), len(br_test.time_series)

(25, 7, 27)

In [4]:
splits = [0.25, 0.5, 0.75, 1.0]
br_train_splits = {}

for pct in splits:
    n_days = len(br_train.time_series)
    train_days = list(br_train.time_series.keys())[-int(pct*n_days):]
    y = pd.concat([
        br_train.time_series[k]['PW_0.5h'] for k in train_days
    ])
    X = pd.concat([
        br_train.time_series[k].drop(
            columns=['timestamp','PW_0.5h','date','time']
        ) for k in train_days
    ])

    # Drop std=0 variables
    X = X[
        [c for c in X.columns if np.std(X[c]) != 0]
    ]

    if pct == 0.25:
        keep_cols = X.columns
        
    br_train_splits.update(
        {
            str(pct):
            {
                'X': X.values,
                'y': y.values,
                'cols': keep_cols.tolist()
            }
        }
    )
    print(f"{pct}\t-\t{X.shape}\t-\t{y.shape}\t-\n{keep_cols.tolist()}\n")

0.25	-	(14512, 12)	-	(14512,)	-
['Product temperature base [°C]', 'Evaportator temperature base [°C]', 'Power supply [V]', 'Condenser temperature base [°C]', 'Instant power consumption [W]', 'Signal [DBM]', 'Door_alert', 'Door_close', 'Door_open', 'Machine_cooling', 'Machine_defrost', 'Machine_pause']

0.5	-	(29695, 12)	-	(29695,)	-
['Product temperature base [°C]', 'Evaportator temperature base [°C]', 'Power supply [V]', 'Condenser temperature base [°C]', 'Instant power consumption [W]', 'Signal [DBM]', 'Door_alert', 'Door_close', 'Door_open', 'Machine_cooling', 'Machine_defrost', 'Machine_pause']

0.75	-	(44878, 12)	-	(44878,)	-
['Product temperature base [°C]', 'Evaportator temperature base [°C]', 'Power supply [V]', 'Condenser temperature base [°C]', 'Instant power consumption [W]', 'Signal [DBM]', 'Door_alert', 'Door_close', 'Door_open', 'Machine_cooling', 'Machine_defrost', 'Machine_pause']

1.0	-	(60166, 12)	-	(60166,)	-
['Product temperature base [°C]', 'Evaportator temperature

In [5]:
br_dev_data = {
    'X': {dt: x[keep_cols].values for dt, x in br_dev.time_series.items()},
    'y': {dt: x['PW_0.5h'].values for dt, x in br_dev.time_series.items()}
}
br_test_data = {
    'X': {dt: x[keep_cols].values for dt, x in br_test.time_series.items()},
    'y': {dt: x['PW_0.5h'].values for dt, x in br_test.time_series.items()}
}

In [6]:
model = DeepStoppingModel(
    'transformer',
    12
)

config.json:   0%|          | 0.00/248 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

KeyboardInterrupt: 